In [1]:
import torch.nn as nn
import torch

In [2]:
embedding = nn.Embedding(100, 5)

In [5]:
word_indices = torch.tensor([[0, 12, 35], [14, 25, 32]])
embeds = embedding(word_indices)
print(embeds)

tensor([[[-1.3503,  0.3081, -1.2628, -0.9445, -0.5948],
         [ 0.0323, -0.2842,  1.0266,  0.8103, -0.6095],
         [-0.7855,  0.2334,  0.5135,  0.2064,  0.4160]],

        [[ 0.1088, -0.5876, -0.9239, -1.1575, -0.5690],
         [-0.1505, -0.7596, -0.1152, -0.8817, -0.6129],
         [ 2.1962,  1.0490, -0.1183, -0.4803,  0.7735]]],
       grad_fn=<EmbeddingBackward0>)


In [1]:
from SingleHeadAttention import SingleHeadAttention
import torch

# Create a simple single head attention layer:
embed_dim = 10
query_dim = 5

single_headed_attn = SingleHeadAttention(embed_dim, query_dim)

# Create random source and target sequences:
batch_size = 32
src_seq_length    = 16
target_seq_length = 8

src_seq    = torch.randn(batch_size, src_seq_length, embed_dim)
target_seq = torch.randn(batch_size, target_seq_length, embed_dim)

# Try out the forward operation of the layer:
attn_output = single_headed_attn(src_seq, target_seq)

print(attn_output.shape)

torch.Size([32, 8, 10])


In [2]:
from torch.nn import MultiheadAttention
import torch

# Create a simple multi-head attention layer:
embed_dim = 512
key_dim   = 32
value_dim = 64
num_heads = 8

multi_headed_attn = MultiheadAttention(embed_dim, num_heads, 0.25, False, False,batch_first=True)

# Create inputs for the multi-head attn layer:
# Create random source and target sequences:
batch_size = 32
src_seq_length    = 16
target_seq_length = 8

src_seq    = torch.randn(batch_size, src_seq_length, embed_dim)
target_seq = torch.randn(batch_size, target_seq_length, embed_dim)

outputs, output_weights = multi_headed_attn(target_seq, src_seq, src_seq) 

print(outputs.shape)
print(output_weights.shape)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (512x512 and 32x512)

In [5]:
# Test the mask generation logic:
DEVICE = 'cpu'
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask


tgt_mask = generate_square_subsequent_mask(8)

print(tgt_mask)

tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0.]])


In [14]:
PAD_IDX = 10

def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

src = torch.randint((8, 2, 1), 0, 20)
tgt = torch.randint((10, 2, 1), 0, 20)
src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt)

print(src_padding_mask)

TypeError: randint() received an invalid combination of arguments - got (tuple, int, int), but expected one of:
 * (int high, tuple of ints size, *, torch.Generator generator, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (int high, tuple of ints size, *, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (int low, int high, tuple of ints size, *, torch.Generator generator, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (int low, int high, tuple of ints size, *, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
